In [1]:
import os, sys
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

In [2]:
name = 'colorado_headwaters'
if name == 'colorado_headwaters':
    HUC8_ID = '14010001'
elif name == 'upper_san_juan':
    HUC8_ID = '14080101'

In [3]:
## load list of dates
df = pd.read_csv('../data/SNOTEL_80th_perc_WY2023_{0}'.format(name))
df

,time,value,SWE_diff
0,2022-10-27,1.032258,0.425806
1,2022-12-10,3.277419,0.696774
2,2022-12-24,5.003226,0.729032
3,2022-12-25,6.119355,1.116129
4,2022-12-27,6.677419,0.467742
5,2022-12-28,7.067742,0.390323
6,2022-12-29,7.438710,0.370968
7,2022-12-31,8.067742,0.406452
8,2023-01-01,8.416129,0.548387
9,2023-01-11,10.074194,0.390323


In [4]:
## get centroid lat lon for HUC8 watershed

fname = '/data/projects/Comet/cwp140/preprocessed/PRISM/PRISM_HUC8_CO.nc'
PRISM = xr.open_dataset(fname)
PRISM = PRISM.sel(HUC8=HUC8_ID)
lat = PRISM.lat.values
lon = PRISM.lon.values

df['lat'] = lat
df['lon'] = lon

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/eaton-scripps/share/proj failed


In [5]:
df

,time,value,SWE_diff,lat,lon
0,2022-10-27,1.032258,0.425806,40.003075,-106.411878
1,2022-12-10,3.277419,0.696774,40.003075,-106.411878
2,2022-12-24,5.003226,0.729032,40.003075,-106.411878
3,2022-12-25,6.119355,1.116129,40.003075,-106.411878
4,2022-12-27,6.677419,0.467742,40.003075,-106.411878
5,2022-12-28,7.067742,0.390323,40.003075,-106.411878
6,2022-12-29,7.438710,0.370968,40.003075,-106.411878
7,2022-12-31,8.067742,0.406452,40.003075,-106.411878
8,2023-01-01,8.416129,0.548387,40.003075,-106.411878
9,2023-01-11,10.074194,0.390323,40.003075,-106.411878


In [6]:
pressure_lst = []
pressure_lst_50 = []
pressure_lst_100 = []
for i, time in enumerate(df['time'].values):
    datetime = pd.to_datetime(time, format='%Y-%m-%d')
    year = datetime.year
    month = datetime.strftime('%m')
    fname = '/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/{0}/{0}{1}.nc'.format(year, month)
    print(fname)
    try: 
        ds = xr.open_dataset(fname)
        pressure = ds.sp.sel(time=datetime, latitude=lat, longitude=360+lon, method='nearest')
        if pressure.size > 1:
            pres = pressure.values[0] / 100. # convert to hPa
        else:
            pres = pressure.values / 100. # convert to hPa
        
    except FileNotFoundError:
        pres = 700. # convert to hPa
    
    
    pressure_lst.append(pres)
    pressure_lst_50.append(pres-50)
    pressure_lst_100.append(pres-100)

/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2022/202210.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2022/202212.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2022/202212.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2022/202212.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2022/202212.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2022/202212.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2022/202212.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2022/202212.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2023/202301.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmospheric_model/analysis/surface/2023/202301.nc
/data/projects/40YearReanalysis/Forcing/ERA5/atmos

In [7]:
df['pres'] = pressure_lst
df['pres_50'] = pressure_lst_50
df['pres_100'] = pressure_lst_100
df

,time,value,SWE_diff,lat,lon,pres,pres_50,pres_100
0,2022-10-27,1.032258,0.425806,40.003075,-106.411878,733.322031,683.322031,633.322031
1,2022-12-10,3.277419,0.696774,40.003075,-106.411878,700.000000,650.000000,600.000000
2,2022-12-24,5.003226,0.729032,40.003075,-106.411878,700.000000,650.000000,600.000000
3,2022-12-25,6.119355,1.116129,40.003075,-106.411878,700.000000,650.000000,600.000000
4,2022-12-27,6.677419,0.467742,40.003075,-106.411878,700.000000,650.000000,600.000000
5,2022-12-28,7.067742,0.390323,40.003075,-106.411878,700.000000,650.000000,600.000000
6,2022-12-29,7.438710,0.370968,40.003075,-106.411878,700.000000,650.000000,600.000000
7,2022-12-31,8.067742,0.406452,40.003075,-106.411878,700.000000,650.000000,600.000000
8,2023-01-01,8.416129,0.548387,40.003075,-106.411878,729.821250,679.821250,629.821250
9,2023-01-11,10.074194,0.390323,40.003075,-106.411878,734.125859,684.125859,634.125859


In [8]:
len(df)

15

In [9]:
lev_lst = [1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 700, 650, 600, 550, 500, 450, 400, 350, 300, 250, 225, 200, 175, 150, 125, 100, 70, 50, 30, 20, 10, 7, 5, 3, 2, 1]


for i, row in enumerate(range(len(df))):
    test1 = df['pres_50'].iloc[i]
    test2 = df['pres_100'].iloc[i]

    print([x for x in lev_lst if test2 < x < test1])

[650]
[]
[]
[]
[]
[]
[]
[]
[650]
[650]
[650]
[650]
[650]
[650]
[650]
